In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("FlavorSense.csv")

In [3]:
data

,age,sleep_cycle,exercise_habits,climate_zone,historical_cuisine_exposure,preferred_taste
0,56.0,Irregular,Heavy,Temperate,NaN,Salty
1,NaN,Night Owl,Heavy,Temperate,Asian,Sweet
2,46.0,Night Owl,Heavy,Cold,Mixed,Sour
3,32.0,Early Bird,Heavy,Cold,Mediterranean,Salty
4,60.0,Night Owl,Moderate,Dry,Mediterranean,Sour
...,...,...,...,...,...,...
9995,55.0,Irregular,Light,Cold,Mixed,Sour
9996,51.0,Early Bird,Light,Dry,Asian,Sweet
9997,57.0,Early Bird,NaN,Cold,Mediterranean,Salty
9998,64.0,Irregular,Light,Tropical,Mixed,Sweet


In [4]:
data.head()


,age,sleep_cycle,exercise_habits,climate_zone,historical_cuisine_exposure,preferred_taste
0,56.0,Irregular,Heavy,Temperate,NaN,Salty
1,NaN,Night Owl,Heavy,Temperate,Asian,Sweet
2,46.0,Night Owl,Heavy,Cold,Mixed,Sour
3,32.0,Early Bird,Heavy,Cold,Mediterranean,Salty
4,60.0,Night Owl,Moderate,Dry,Mediterranean,Sour


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   age                          9032 non-null   float64
 1   sleep_cycle                  9316 non-null   object 
 2   exercise_habits              9196 non-null   object 
 3   climate_zone                 9467 non-null   object 
 4   historical_cuisine_exposure  9346 non-null   object 
 5   preferred_taste              10000 non-null  object 
dtypes: float64(1), object(5)
memory usage: 468.9+ KB


In [6]:
data.describe()

,age
count,9032.000000
mean,43.552702
std,14.889327
min,18.000000
25%,31.000000
50%,43.000000
75%,56.000000
max,69.000000


In [7]:
data.nunique()

age                            52
sleep_cycle                     3
exercise_habits                 3
climate_zone                    4
historical_cuisine_exposure     3
preferred_taste                 4
dtype: int64

In [8]:
data.dtypes

age                            float64
sleep_cycle                     object
exercise_habits                 object
climate_zone                    object
historical_cuisine_exposure     object
preferred_taste                 object
dtype: object

In [9]:
data.isnull().sum()

age                            968
sleep_cycle                    684
exercise_habits                804
climate_zone                   533
historical_cuisine_exposure    654
preferred_taste                  0
dtype: int64

In [10]:
#Eksikleri Doldurma İşi

In [11]:
data["sleep_cycle"] = data["sleep_cycle"].fillna(data["sleep_cycle"].mode()[0])
data["exercise_habits"] = data["exercise_habits"].fillna(data["exercise_habits"].mode()[0])
data["climate_zone"] = data["climate_zone"].fillna(data["climate_zone"].mode()[0])
data["historical_cuisine_exposure"] = data["historical_cuisine_exposure"].fillna(data["historical_cuisine_exposure"].mode()[0])

In [12]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [13]:
#imputer = IterativeImputer(random_state = 0) , eksik olan her sayısal sütunu diğer sütunları kullanarak makine öğrenmesiyle tahmin eder.
#random_state=0 yazmak, sonucun tekrar üretilebilir olmasını sağlar.
#fit_transform() yöntemi:
#fit: diğer sütunlar arasındaki ilişkileri öğrenir.
#transform: eksik değerleri doldurur.
#full_sayisal adlı doldurulmuş NumPy array’i,
#Orijinal data DataFrame’ine tekrar yazarız.
#Bu sayede eksik veriler tahmin edilmiş ve data’ya geri yerleştirilmiş olur.

In [14]:
numerical_data = data.select_dtypes(include = ["float64"])
imputer = IterativeImputer(random_state=0)
full_numeric = imputer.fit_transform(numerical_data)
data[numerical_data.columns] = full_numeric

In [15]:
data.isnull().sum()

age                            0
sleep_cycle                    0
exercise_habits                0
climate_zone                   0
historical_cuisine_exposure    0
preferred_taste                0
dtype: int64

In [16]:
#LABELENCODER
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [17]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data["preferred_taste"])
X = data.drop("preferred_taste",axis =1 )

In [18]:
# Boşlukları Doldurduktan Sonra Kategorik olanları ve Numeric olanları ayırıyoruz

In [19]:
kategorik_cols = X.select_dtypes(include = ["object"]).columns.tolist()
numeric_col = X.select_dtypes(include = ["float64"]).columns

In [20]:
#OneHotEncoder'a sadece X içindeki kategorikler gider, y zaten encode edilmiş olur

In [21]:
#oneHotEncoder + ColumnTransformer

In [40]:
column_transformer = ColumnTransformer(
    transformers = [
        ("cat",OneHotEncoder(drop = "first"),kategorik_cols)
         ],
    remainder = "passthrough"
)
X_processed = column_transformer.fit_transform(X)

In [41]:
#DataFrame'e dönüştürmek

In [46]:
#Sütun isimlerini almak
ohe = column_transformer.named_transformers_["cat"]
encoded_col_names = ohe.get_feature_names_out(kategorik_cols)
#sayısal sütün
numeric_data = X[numeric_col].reset_index(drop = True)
## Yeni DataFrame
X_encoded_df = pd.DataFrame(X_processed, columns=list(encoded_col_names) + list(numeric_col))
X_encoded_df

,sleep_cycle_Irregular,sleep_cycle_Night Owl,exercise_habits_Light,exercise_habits_Moderate,climate_zone_Dry,climate_zone_Temperate,climate_zone_Tropical,historical_cuisine_exposure_Mediterranean,historical_cuisine_exposure_Mixed,age
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,56.000000
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,43.552702
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,46.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,32.000000
4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,60.000000
...,...,...,...,...,...,...,...,...,...,...
9995,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,55.000000
9996,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,51.000000
9997,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,57.000000
9998,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,64.000000


In [72]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [73]:
X_train,X_test,y_train,y_test = train_test_split(
    X_encoded_df,y, test_size=0.2, random_state=42
)

In [76]:
model = RandomForestClassifier()

In [77]:
model.fit(X_train ,y_train)

RandomForestClassifier()

In [78]:
y_pred = model.predict(X_test)
y_pred_labels = label_encoder.inverse_transform(y_pred)
print(y_pred_labels)

['Spicy' 'Sour' 'Sweet' ... 'Sour' 'Sweet' 'Salty']
